In [1]:
import chromadb

client = chromadb.PersistentClient("aggr")

col = client.get_or_create_collection("test")

col.add(ids=[str(i) for i in range(1)],documents=["doc 1"],metadatas=[{"a":"1","b":"2"}])

Add of existing embedding ID: 0
Add of existing embedding ID: 0
Add of existing embedding ID: 0
Add of existing embedding ID: 0
Add of existing embedding ID: 0
Add of existing embedding ID: 0
Add of existing embedding ID: 0
Add of existing embedding ID: 0
Add of existing embedding ID: 0
Insert of existing embedding ID: 0
Add of existing embedding ID: 0


In [2]:
col.get()

SELECT "embeddings"."id","embeddings"."embedding_id","embeddings"."seq_id",MAX(CASE WHEN embedding_metadata.key = 'chroma:document' THEN embedding_metadata.string_value END) AS document_content,'{' || GROUP_CONCAT(DISTINCT CASE WHEN embedding_metadata.key <> 'chroma:document' THEN '"' || embedding_metadata.key || '": ' || CASE
                                                                                                             WHEN embedding_metadata.string_value IS NOT NULL
                                                                                                                 THEN '"' || replace(embedding_metadata.string_value, '"','"')|| '"'
                                                                                                             WHEN embedding_metadata.int_value IS NOT NULL
                                                                                                                 THEN embedding_metadata.int_value
                              

{'ids': ['0'],
 'embeddings': None,
 'metadatas': [{'a': '1', 'b': '2'}],
 'documents': [None],
 'uris': None,
 'data': None}

In [ ]:
from typing import Any
from pypika.utils import builder
from pypika.terms import Function


class CustomMax(Function):
    is_aggregate = True

    def __init__(self, name, *args, **kwargs):
        super(CustomMax, self).__init__(name, *args, **kwargs)

        self._filters = []
        self._include_filter = False

    # @builder
    # def filter(self, *filters: Any) -> "AnalyticFunction":
    #     self._include_filter = True
    #     self._filters += filters
    # 
    # def get_filter_sql(self, **kwargs: Any) -> str:
    #     if self._include_filter:
    #         return "WHERE {criterions}".format(criterions=Criterion.all(self._filters).get_sql(**kwargs))

    def get_function_sql(self, **kwargs: Any):
        sql = super(CustomMax, self).get_function_sql(**kwargs)

        return sql

custom_max = CustomMax("max", "column")

print(custom_max.get_sql())

In [ ]:
from pypika import CustomFunction


class MyCustomFunction(CustomFunction):
    def __init__(self, function_name, params, alias=None):
        super().__init__(function_name, params)
        self.function_name = function_name
        self._alias = alias
        
    def get_function_sql(self, **kwargs: Any):
        sql = f"{self.function_name}({self.params})"
        if self._alias:
            sql += f" AS {self._alias}"
        return sql
    
    def _as(self, alias):
        self._alias = alias
    
print(MyCustomFunction("max", "CASE WHEN emd.key = 'chroma:document' THEN emd.string_value END", "document_content").get_function_sql())
    


In [ ]:
class CustomStringFunction(CustomFunction):
    def __init__(self, params, alias=None):
        super().__init__("CustomStringFunction", params)
        self._alias = alias

    def get_function_sql(self, **kwargs: Any):
        return f"{self.params}"

    def _as(self, alias):
        self._alias = alias

print(CustomStringFunction("""'{' || GROUP_CONCAT(DISTINCT CASE WHEN emd.key <> 'chroma:document' THEN '"' || emd.key || '": ' || CASE
                                                                                                             WHEN emd.string_value IS NOT NULL
                                                                                                                 THEN '"' || replace(emd.string_value, '"','\"')|| '"'
                                                                                                             WHEN emd.int_value IS NOT NULL
                                                                                                                 THEN emd.int_value
                                                                                                             WHEN float_value IS NOT NULL
                                                                                                                 THEN float_value
                                                                                                             ELSE bool_value
        END END) || '}'""",alias="metadata").get_function_sql())